In [1]:
pip install numpy matplotlib reportlab


  Using cached reportlab-4.4.5-py3-none-any.whl.metadata (1.7 kB)
Using cached reportlab-4.4.5-py3-none-any.whl (2.0 MB)
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import tkinter as tk
from tkinter import ttk, messagebox
import numpy as np
import matplotlib.pyplot as plt
import math
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image
from reportlab.lib.pagesizes import letter
from reportlab.lib.styles import getSampleStyleSheet

# -----------------------
# Utility: normal PDF/CDF and inverse CDF (via binary search)
# -----------------------
TWOPI = 2 * math.pi

def norm_pdf(x, mu, sigma):
    return math.exp(-0.5 * ((x - mu) / sigma) ** 2) / (sigma * math.sqrt(TWOPI))

def norm_cdf(x, mu, sigma):
    # using erf
    return 0.5 * (1 + math.erf((x - mu) / (sigma * math.sqrt(2))))

def norm_ppf(p, mu, sigma, tol=1e-8, max_iter=100):
    """Inverse CDF: numeric binary search (robust, no scipy)."""
    if not (0 < p < 1):
        if p == 0: return -math.inf
        if p == 1: return math.inf
        raise ValueError("p must be in (0,1)")
    lo = mu - 10 * sigma
    hi = mu + 10 * sigma
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        v = norm_cdf(mid, mu, sigma)
        if abs(v - p) < tol:
            return mid
        if v < p:
            lo = mid
        else:
            hi = mid
    return 0.5 * (lo + hi)

# -----------------------
# Problem-specific compute + plot functions
# -----------------------
def plot_normal_and_shade(mu, sigma, shade_region, filename, title, xstep=None, annotate=None):
    """shade_region: tuple (shade_type, params)
       shade_type: 'left' -> shade x < param
                   'right' -> shade x > param
                   'between' -> shade param = (a, b)
                   'middle' -> shade param = (a, b) for center shading
    """
    if xstep is None:
        xstep = max(0.1, sigma/10.0)
    x_min = mu - 4*sigma
    x_max = mu + 4*sigma
    x = np.arange(x_min, x_max + xstep, xstep)
    y = [norm_pdf(xx, mu, sigma) for xx in x]

    plt.figure(figsize=(7,4))
    plt.plot(x, y, linewidth=1.8)
    plt.title(title)
    plt.xlabel("x")
    plt.ylabel("PDF")
    # shading
    if shade_region[0] == 'left':
        a = shade_region[1]
        xs = np.extract(x <= a, x)
        ys = np.extract(x <= a, y)
        plt.fill_between(xs, ys, alpha=0.5)
    elif shade_region[0] == 'right':
        a = shade_region[1]
        xs = np.extract(x >= a, x)
        ys = np.extract(x >= a, y)
        plt.fill_between(xs, ys, alpha=0.5)
    elif shade_region[0] == 'between' or shade_region[0] == 'middle':
        a, b = shade_region[1]
        xs = np.extract((x >= a) & (x <= b), x)
        ys = np.extract((x >= a) & (x <= b), y)
        plt.fill_between(xs, ys, alpha=0.5)
    # annotate mean & special lines if provided
    if annotate:
        for ann in annotate:
            plt.axvline(ann, linestyle='--', alpha=0.8)
    plt.grid(axis='y', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.savefig(filename, dpi=150)
    plt.close()

def compute_all():
    """Compute problems 1-5, produce graphs, and return a long text report (string)."""
    lines = []
    lines.append("Student Activity: Normal Distribution Applications in Engineering\n")
    lines.append("Computations and graphs produced programmatically.\n")

    # -------- Problem 1 (Concrete Strength) ----------
    lines.append("Problem 1 — Civil Engineering (Concrete Strength)")
    mu1, sd1, n1 = 35.0, 5.0, 400
    xq = 30.0
    p_lt_30 = norm_cdf(xq, mu1, sd1)
    expected_lt_30 = p_lt_30 * n1
    z1 = (xq - mu1) / sd1
    lines.append(f"Mean = {mu1} MPa, SD = {sd1} MPa, n = {n1}")
    lines.append(f"1. P(X < {xq}) = {p_lt_30:.6f}  (z = {z1:.4f})")
    lines.append(f"2. Expected count below {xq}: {expected_lt_30:.2f} cylinders")
    lines.append("3. Bell curve graph saved as 'activity1_graph.png'")
    lines.append("4. Interpretation: If a non-negligible fraction of cylinders fall below 30 MPa,")
    lines.append("   quality control must adjust mix design or production to reduce weak units.\n")
    # graph: shade left of 30
    plot_normal_and_shade(mu1, sd1, ('left', xq), "activity1_graph.png",
                          f"Concrete strength ~ N({mu1}, {sd1}^2) — P(X<{xq}) shaded", xstep=0.1,
                          annotate=[mu1, xq])

    # -------- Problem 2 (Resistor Tolerance) ----------
    lines.append("Problem 2 — Electronics Engineering (Resistor Tolerance)")
    mu2, sd2, N2 = 100.0, 3.0, 10000
    xq2 = 106.0
    p_gt_106 = 1 - norm_cdf(xq2, mu2, sd2)
    expected_gt_106 = p_gt_106 * N2
    z2 = (xq2 - mu2) / sd2
    lines.append(f"Mean = {mu2} Ω, SD = {sd2} Ω, production lot = {N2}")
    lines.append(f"1. P(X > {xq2}) = {p_gt_106:.6f}  (z = {z2:.4f})")
    lines.append(f"2. Expected count above {xq2}: {expected_gt_106:.1f} resistors")
    lines.append("3. Graph saved as 'activity2_graph.png'")
    lines.append("4. Interpretation: A small tail above tolerance reduces yield and increases scrap or rework.\n")
    plot_normal_and_shade(mu2, sd2, ('right', xq2), "activity2_graph.png",
                          f"Resistor resistance ~ N({mu2}, {sd2}^2) — P(X>{xq2}) shaded", xstep=0.05,
                          annotate=[mu2, mu2+2*sd2, xq2])

    # -------- Problem 3 (CPU Temperature) ----------
    lines.append("Problem 3 — Computer Engineering (CPU Temperature)")
    mu3, sd3 = 70.0, 5.0
    top_pct = 0.05
    cutoff_top5 = norm_ppf(1 - top_pct, mu3, sd3)
    z3 = (cutoff_top5 - mu3) / sd3
    lines.append(f"Mean = {mu3} °C, SD = {sd3} °C")
    lines.append(f"1. Cutoff separating top 5% (i.e., 95th percentile) = {cutoff_top5:.4f} °C (z = {z3:.4f})")
    lines.append("2. Graph saved as 'activity3_graph.png'")
    lines.append("3. Interpretation: The 95th percentile guides cooling requirement — designs should handle")
    lines.append("   temperatures up to this cutoff to maintain reliability.\n")
    plot_normal_and_shade(mu3, sd3, ('right', cutoff_top5), "activity3_graph.png",
                          f"CPU Temp ~ N({mu3}, {sd3}^2) — Top 5% shaded", xstep=0.1,
                          annotate=[mu3, cutoff_top5])

    # -------- Problem 4 (Bridge Load Testing) ----------
    lines.append("Problem 4 — Civil Engineering (Bridge Load Testing)")
    mu4, sd4 = 80.0, 10.0
    middle_pct = 0.70
    lower_q = (1 - middle_pct) / 2
    upper_q = 1 - lower_q
    lower_val = norm_ppf(lower_q, mu4, sd4)
    upper_val = norm_ppf(upper_q, mu4, sd4)
    lines.append(f"Mean = {mu4} tons, SD = {sd4} tons")
    lines.append(f"1. Middle 70% interval: lower = {lower_val:.4f} tons, upper = {upper_val:.4f} tons (quantiles {lower_q:.2f}, {upper_q:.2f})")
    lines.append("2. Graph saved as 'activity4_graph.png'")
    lines.append("3. Interpretation: Focusing on the middle 70% helps examine typical beams; extremes may be rare but should be tested separately.\n")
    plot_normal_and_shade(mu4, sd4, ('between', (lower_val, upper_val)), "activity4_graph.png",
                          f"Bridge load ~ N({mu4}, {sd4}^2) — Middle {int(middle_pct*100)}% shaded", xstep=0.2,
                          annotate=[mu4, lower_val, upper_val])

    # -------- Problem 5 (Battery Life) ----------
    lines.append("Problem 5 — Electronics Engineering (Battery Life)")
    mu5, sd5, N5 = 500.0, 60.0, 5000
    a5, b5 = 440.0, 560.0
    p_between = norm_cdf(b5, mu5, sd5) - norm_cdf(a5, mu5, sd5)
    expected_between = p_between * N5
    lines.append(f"Mean = {mu5} cycles, SD = {sd5} cycles, lot size = {N5}")
    lines.append(f"1. P({a5} ≤ X ≤ {b5}) = {p_between:.6f}")
    lines.append(f"2. Expected count in range [{a5}, {b5}]: {expected_between:.1f} batteries")
    lines.append("3. Graph saved as 'activity5_graph.png'")
    lines.append("4. Interpretation: The proportion within [440,560] measures typical product reliability and helps set warranties.\n")
    plot_normal_and_shade(mu5, sd5, ('between', (a5, b5)), "activity5_graph.png",
                          f"Battery life ~ N({mu5}, {sd5}^2) — Range {a5}-{b5} shaded", xstep=1.0,
                          annotate=[mu5, a5, b5])

    return "\n".join(lines)

# -----------------------
# PDF generation
# -----------------------
def generate_pdf_report(text, outname="Normal_Distribution_Report.pdf"):
    doc = SimpleDocTemplate(outname, pagesize=letter)
    styles = getSampleStyleSheet()
    flow = []
    for line in text.split("\n"):
        # skip empty lines to avoid many tiny paragraphs
        if line.strip() == "":
            flow.append(Spacer(1,6))
        else:
            flow.append(Paragraph(line.replace("&", "&amp;"), styles["Normal"]))
            flow.append(Spacer(1,6))
    # Append images (in sensible order)
    imgs = [
        ("activity1_graph.png", 400, 240),
        ("activity2_graph.png", 400, 240),
        ("activity3_graph.png", 400, 240),
        ("activity4_graph.png", 400, 240),
        ("activity5_graph.png", 400, 240),
    ]
    for fn, w, h in imgs:
        try:
            flow.append(Spacer(1,12))
            flow.append(Image(fn, width=w, height=h))
        except Exception as e:
            flow.append(Paragraph(f"(Could not embed image {fn}: {e})", styles["Normal"]))
    doc.build(flow)

# -----------------------
# Tkinter UI
# -----------------------
def run_app():
    root = tk.Tk()
    root.title("Normal Distribution Activities (Compute All)")
    root.geometry("900x650")

    frame = ttk.Frame(root, padding=10)
    frame.pack(fill="both", expand=True)

    # Title
    title = ttk.Label(frame, text="Normal Distribution — Activities 1 to 5", font=("Segoe UI", 14, "bold"))
    title.pack(pady=(0,10))

    # Text area
    text_frame = ttk.Frame(frame)
    text_frame.pack(fill="both", expand=True)
    text = tk.Text(text_frame, wrap="word", font=("Consolas", 10))
    text.pack(side="left", fill="both", expand=True)
    scroll = ttk.Scrollbar(text_frame, command=text.yview)
    scroll.pack(side="right", fill="y")
    text['yscrollcommand'] = scroll.set

    # Buttons
    btn_frame = ttk.Frame(frame)
    btn_frame.pack(fill="x", pady=8)

    def on_compute_all():
        text.delete("1.0", tk.END)
        text.insert(tk.END, "Computing... (this will generate graphs as PNG files)\n")
        root.update()
        report = compute_all()
        text.delete("1.0", tk.END)
        text.insert(tk.END, report)
        messagebox.showinfo("Done", "Computations complete. Graph PNG files saved (activity1_graph.png ... activity5_graph.png).")

    def on_generate_pdf():
        content = text.get("1.0", tk.END).strip()
        if not content:
            messagebox.showwarning("No content", "Please compute first (Compute All) before generating the PDF.")
            return
        try:
            generate_pdf_report(content, outname="Normal_Distribution_Report.pdf")
            messagebox.showinfo("PDF saved", "PDF saved as 'Normal_Distribution_Report.pdf'")
        except Exception as e:
            messagebox.showerror("PDF error", f"Failed to create PDF: {e}")

    compute_btn = ttk.Button(btn_frame, text="Compute All Activities (1–5)", command=on_compute_all)
    compute_btn.pack(side="left", padx=8)

    pdf_btn = ttk.Button(btn_frame, text="Generate PDF Report", command=on_generate_pdf)
    pdf_btn.pack(side="left", padx=8)

    quit_btn = ttk.Button(btn_frame, text="Quit", command=root.destroy)
    quit_btn.pack(side="right", padx=8)

    root.mainloop()

if __name__ == "__main__":
    run_app()
